In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import pandas as pd
pd.set_option('display.max_columns', None)
from time import sleep
from selenium.common.exceptions import NoSuchElementException

from selenium.webdriver.firefox.options import Options

firefox_options = Options()
firefox_options.add_argument("--headless")

url_main = 'https://www.flashscore.com'

tourney_list = ['indian-wells-2023',]

for tourney in tourney_list:
    tourney_results = url_main + '/tennis/atp-singles/' + tourney + '/results/'
    browser1 = webdriver.Firefox(options=firefox_options)
    browser1.get(tourney_results)
    sleep(5)
    filename = tourney_results.split('/')[5].strip()+'.csv'
    print(filename)
    
    matches = browser1.find_elements(By.CLASS_NAME, 'event__match')

    matches_data = []

    #matches_df = pd.DataFrame()

    columns_df = ['Aces_left', 'Double Faults_left', '1st Serve Percentage_left', 
                  '1st Serve Points Won_left', '2nd Serve Points Won_left', 
                  'Break Points Converted_left', 'Return Points Won_left', 
                  'Total Points Won_left', 'Total Games Won_left',
                  'Aces_right', 'Double Faults_right', '1st Serve Percentage_right', 
                  '1st Serve Points Won_right', '2nd Serve Points Won_right',
                  'Break Points Converted_right', 'Return Points Won_right', 
                  'Total Points Won_right', 'Total Games Won_right',
                  'player_left', 'player_right', 'date', 'match_result', 
                  'match_info', 'odds_left', 'odds_right']

    for match_id in matches:
        browser = webdriver.Firefox(options=firefox_options)
        browser.get(url_main + '/match/' + match_id.get_attribute('id')[4:] + '/#/match-summary/match-statistics/0')

        sleep(5)

        stats_tab = browser.find_elements(By.XPATH, '//a[@href="#/match-summary/match-statistics"]')
        if len(stats_tab) > 0:

            left = []
            stats_left = browser.find_elements(By.CLASS_NAME, 'stat__homeValue')
            for stat in stats_left:
                left.append(stat.text)

            cols1 = []
            cols2 = []
            col_names = browser.find_elements(By.CLASS_NAME, 'stat__categoryName')
            for col in col_names:
                cols1.append(col.text + '_left')
                cols2.append(col.text + '_right')    

            right = []
            stats_right = browser.find_elements(By.CLASS_NAME, 'stat__awayValue')
            for stat in stats_right:
                right.append(stat.text)

            players = []
            player_names = browser.find_elements(By.CLASS_NAME, 'participant__participantLink')
            for player in player_names:
                players.append(player.get_attribute('href').split('/')[4].strip())


            match_date_time = browser.find_element(By.CLASS_NAME, 'duelParticipant__startTime').text

            match_result = browser.find_element(By.CLASS_NAME, 'duelParticipant__score').text
            #sets = list(list(match_result)[i] for i in [0,4])

            tourney_name = browser.find_element(By.CLASS_NAME, 'tournamentHeader__country').text

            odds = [] 
            odds_elem = browser.find_elements(By.CLASS_NAME, 'oddsValue')
            if len(odds_elem) > 0:
                for odd in odds_elem:
                    odds.append(odd.text)
            else:
                odds = ['', '']

            cols3 = ['player_left', 'player_right', 'date', 'match_result', 'match_info', 'odds_left', 'odds_right']
            cols = cols1 + cols2 + cols3


            match_data = left + right + players + [match_date_time] + [match_result] + [tourney_name] + odds

            match_df = pd.DataFrame(match_data).T
            match_df.columns = cols


            match_df_filtered = match_df[match_df.columns.intersection(columns_df)]

            match_data_filtered = match_df_filtered.values.flatten().tolist()

            matches_data.append(match_data_filtered)

            sleep(3)

            print(match_data)
            print("")
            print(len(match_data))
            print("")
            print(match_data_filtered)
            print("")

        browser.close()
    df = pd.DataFrame(matches_data)
    df.columns = columns_df
    df.to_csv(filename)
    
    browser1.close()

indian-wells-2023.csv
['1', '0', '76%', '81% (30/37)', '58% (7/12)', '0% (0/0)', '39% (12/31)', '59% (10/17)', '100% (3/3)', '19', '10', '77% (10/13)', '12', '76% (37/49)', '46% (22/48)', '61% (59/97)', '5', '100% (9/9)', '38% (3/8)', '71% (12/17)', '0', '2', '65%', '61% (19/31)', '41% (7/17)', '0% (0/3)', '19% (7/37)', '42% (5/12)', '0% (0/0)', '4', '14', '53% (8/15)', '4', '54% (26/48)', '24% (12/49)', '39% (38/97)', '1', '63% (5/8)', '0% (0/9)', '29% (5/17)', 'alcaraz-garfia-carlos', 'medvedev-daniil', '19.03.2023 19:10', '2\n-\n0\nFINISHED', 'ATP - SINGLES: INDIAN WELLS (USA), HARD - FINAL', '1.73', '2.10']

47

['1', '0', '76%', '81% (30/37)', '58% (7/12)', '100% (3/3)', '46% (22/48)', '61% (59/97)', '71% (12/17)', '0', '2', '65%', '61% (19/31)', '41% (7/17)', '0% (0/0)', '24% (12/49)', '39% (38/97)', '29% (5/17)', 'alcaraz-garfia-carlos', 'medvedev-daniil', '19.03.2023 19:10', '2\n-\n0\nFINISHED', 'ATP - SINGLES: INDIAN WELLS (USA), HARD - FINAL', '1.73', '2.10']

['5', '1', '72%

['3', '0', '74%', '80% (16/20)', '86% (6/7)', '100% (3/3)', '50% (11/22)', '67% (8/12)', '60% (3/5)', '17', '6', '67% (4/6)', '7', '81% (22/27)', '56% (19/34)', '67% (41/61)', '3', '100% (5/5)', '60% (3/5)', '80% (8/10)', '1', '1', '65%', '50% (11/22)', '33% (4/12)', '40% (2/5)', '20% (4/20)', '14% (1/7)', '0% (0/3)', '4', '12', '60% (3/5)', '3', '44% (15/34)', '19% (5/27)', '33% (20/61)', '1', '40% (2/5)', '0% (0/5)', '20% (2/10)', 'alcaraz-garfia-carlos', 'draper-jack', '14.03.2023 22:30', '1\n-\n0\nFINISHED / RETIRED -\nDRAPER J.', 'ATP - SINGLES: INDIAN WELLS (USA), HARD - 1/8-FINALS', '1.30', '3.50']

47

['3', '0', '74%', '80% (16/20)', '86% (6/7)', '60% (3/5)', '56% (19/34)', '67% (41/61)', '80% (8/10)', '1', '1', '65%', '50% (11/22)', '33% (4/12)', '0% (0/3)', '19% (5/27)', '33% (20/61)', '20% (2/10)', 'alcaraz-garfia-carlos', 'draper-jack', '14.03.2023 22:30', '1\n-\n0\nFINISHED / RETIRED -\nDRAPER J.', 'ATP - SINGLES: INDIAN WELLS (USA), HARD - 1/8-FINALS', '1.30', '3.50']

[

['4', '2', '57%', '88% (21/24)', '28% (5/18)', '50% (2/4)', '53% (17/32)', '76% (13/17)', '50% (6/12)', '20', '16', '72% (13/18)', '6', '62% (26/42)', '61% (30/49)', '62% (56/91)', '7', '75% (6/8)', '86% (6/7)', '80% (12/15)', '0', '1', '65%', '47% (15/32)', '24% (4/17)', '50% (6/12)', '13% (3/24)', '72% (13/18)', '50% (2/4)', '8', '14', '44% (4/9)', '6', '39% (19/49)', '38% (16/42)', '38% (35/91)', '2', '14% (1/7)', '25% (2/8)', '20% (3/15)', 'fritz-taylor', 'baez-sebastian', '13.03.2023 18:35', '2\n-\n0\nFINISHED', 'ATP - SINGLES: INDIAN WELLS (USA), HARD - 1/16-FINALS', '1.11', '6.50']

47

['4', '2', '57%', '88% (21/24)', '28% (5/18)', '50% (6/12)', '61% (30/49)', '62% (56/91)', '80% (12/15)', '0', '1', '65%', '47% (15/32)', '24% (4/17)', '50% (2/4)', '38% (16/42)', '38% (35/91)', '20% (3/15)', 'fritz-taylor', 'baez-sebastian', '13.03.2023 18:35', '2\n-\n0\nFINISHED', 'ATP - SINGLES: INDIAN WELLS (USA), HARD - 1/16-FINALS', '1.11', '6.50']

['0', '2', '59%', '80% (24/30)', '38% (8/

['4', '1', '62%', '66% (38/58)', '46% (16/35)', '64% (7/11)', '37% (16/43)', '48% (19/40)', '40% (4/10)', '17', '12', '60% (12/20)', '6', '58% (54/93)', '42% (35/83)', '51% (89/176)', '5', '69% (9/13)', '31% (4/13)', '50% (13/26)', '3', '5', '52%', '63% (27/43)', '53% (21/40)', '60% (6/10)', '34% (20/58)', '54% (19/35)', '36% (4/11)', '35', '27', '86% (12/14)', '6', '58% (48/83)', '42% (39/93)', '49% (87/176)', '5', '69% (9/13)', '31% (4/13)', '50% (13/26)', 'khachanov-karen', 'davidovich-fokina-alejandro', '12.03.2023 17:20', '1\n-\n2\nFINISHED', 'ATP - SINGLES: INDIAN WELLS (USA), HARD - 1/16-FINALS', '1.62', '2.30']

47

['4', '1', '62%', '66% (38/58)', '46% (16/35)', '40% (4/10)', '42% (35/83)', '51% (89/176)', '50% (13/26)', '3', '5', '52%', '63% (27/43)', '53% (21/40)', '36% (4/11)', '42% (39/93)', '49% (87/176)', '50% (13/26)', 'khachanov-karen', 'davidovich-fokina-alejandro', '12.03.2023 17:20', '1\n-\n2\nFINISHED', 'ATP - SINGLES: INDIAN WELLS (USA), HARD - 1/16-FINALS', '1.62

['3', '1', '64%', '65% (39/60)', '47% (16/34)', '83% (10/12)', '35% (12/34)', '63% (20/32)', '57% (4/7)', '22', '25', '71% (22/31)', '8', '59% (55/94)', '48% (32/66)', '54% (87/160)', '3', '80% (8/10)', '40% (4/10)', '60% (12/20)', '3', '1', '52%', '65% (22/34)', '38% (12/32)', '43% (3/7)', '35% (21/60)', '53% (18/34)', '17% (2/12)', '16', '28', '67% (8/12)', '7', '52% (34/66)', '41% (39/94)', '46% (73/160)', '2', '60% (6/10)', '20% (2/10)', '40% (8/20)', 'mannarino-adrian', 'musetti-lorenzo', '11.03.2023 18:10', '2\n-\n0\nFINISHED', 'ATP - SINGLES: INDIAN WELLS (USA), HARD - 1/32-FINALS', '1.91', '1.91']

47

['3', '1', '64%', '65% (39/60)', '47% (16/34)', '57% (4/7)', '48% (32/66)', '54% (87/160)', '60% (12/20)', '3', '1', '52%', '65% (22/34)', '38% (12/32)', '17% (2/12)', '41% (39/94)', '46% (73/160)', '40% (8/20)', 'mannarino-adrian', 'musetti-lorenzo', '11.03.2023 18:10', '2\n-\n0\nFINISHED', 'ATP - SINGLES: INDIAN WELLS (USA), HARD - 1/32-FINALS', '1.91', '1.91']

['2', '1', '56%

...

['3', '2', '64%', '81% (38/47)', '62% (16/26)', '100% (1/1)', '14% (5/37)', '48% (13/27)', '100% (1/1)', '18', '17', '87% (13/15)', '8', '74% (54/73)', '28% (18/64)', '53% (72/137)', '3', '100% (11/11)', '9% (1/11)', '55% (12/22)', '5', '3', '58%', '86% (32/37)', '52% (14/27)', '0% (0/1)', '19% (9/47)', '38% (10/26)', '0% (0/1)', '24', '21', '54% (7/13)', '5', '72% (46/64)', '26% (19/73)', '47% (65/137)', '1', '91% (10/11)', '0% (0/11)', '45% (10/22)', 'kubler-jason', 'sonego-lorenzo', '08.03.2023 15:55', '2\n-\n0\nFINISHED', 'ATP - SINGLES: INDIAN WELLS (USA), HARD - 1/64-FINALS', '2.75', '1.44']

47

['3', '2', '64%', '81% (38/47)', '62% (16/26)', '100% (1/1)', '28% (18/64)', '53% (72/137)', '55% (12/22)', '5', '3', '58%', '86% (32/37)', '52% (14/27)', '0% (0/1)', '26% (19/73)', '47% (65/137)', '45% (10/22)', 'kubler-jason', 'sonego-lorenzo', '08.03.2023 15:55', '2\n-\n0\nFINISHED', 'ATP - SINGLES: INDIAN WELLS (USA), HARD - 1/64-FINALS', '2.75', '1.44']

['1', '2', '59%', '57% (27/4

['3', '0', '78%', '53% (25/47)', '15% (2/13)', '50% (6/12)', '22% (6/27)', '61% (17/28)', '43% (3/7)', '11', '7', '0% (0/3)', '5', '45% (27/60)', '42% (23/55)', '43% (50/115)', '2', '25% (2/8)', '33% (3/9)', '29% (5/17)', '5', '1', '49%', '78% (21/27)', '39% (11/28)', '57% (4/7)', '47% (22/47)', '85% (11/13)', '50% (6/12)', '26', '17', '60% (3/5)', '6', '58% (32/55)', '55% (33/60)', '57% (65/115)', '5', '67% (6/9)', '75% (6/8)', '71% (12/17)', 'muller-alexandre', 'riedi-leandro', '07.03.2023 18:05', '0\n-\n2\nFINISHED', 'ATP - SINGLES: INDIAN WELLS (USA) - QUALIFICATION, HARD - FINAL', '', '']

47

['3', '0', '78%', '53% (25/47)', '15% (2/13)', '43% (3/7)', '42% (23/55)', '43% (50/115)', '29% (5/17)', '5', '1', '49%', '78% (21/27)', '39% (11/28)', '50% (6/12)', '55% (33/60)', '57% (65/115)', '71% (12/17)', 'muller-alexandre', 'riedi-leandro', '07.03.2023 18:05', '0\n-\n2\nFINISHED', 'ATP - SINGLES: INDIAN WELLS (USA) - QUALIFICATION, HARD - FINAL', '', '']

['7', '6', '54%', '69% (35/5

['1', '3', '66%', '50% (20/40)', '48% (10/21)', '60% (6/10)', '42% (13/31)', '37% (7/19)', '33% (1/3)', '11', '23', '44% (4/9)', '7', '49% (30/61)', '40% (20/50)', '45% (50/111)', '2', '43% (3/7)', '14% (1/7)', '29% (4/14)', '2', '3', '62%', '58% (18/31)', '63% (12/19)', '67% (2/3)', '50% (20/40)', '52% (11/21)', '40% (4/10)', '16', '20', '50% (6/12)', '5', '60% (30/50)', '51% (31/61)', '55% (61/111)', '6', '86% (6/7)', '57% (4/7)', '71% (10/14)', 'meligeni-rodrigues-alves-felipe', 'hijikata-rinky', '07.03.2023 01:30', '0\n-\n1\nFINISHED / RETIRED -\nMELIGENI ALVES F.', 'ATP - SINGLES: INDIAN WELLS (USA) - QUALIFICATION, HARD - SEMI-FINALS', '', '']

47

['1', '3', '66%', '50% (20/40)', '48% (10/21)', '33% (1/3)', '40% (20/50)', '45% (50/111)', '29% (4/14)', '2', '3', '62%', '58% (18/31)', '63% (12/19)', '40% (4/10)', '51% (31/61)', '55% (61/111)', '71% (10/14)', 'meligeni-rodrigues-alves-felipe', 'hijikata-rinky', '07.03.2023 01:30', '0\n-\n1\nFINISHED / RETIRED -\nMELIGENI ALVES F.',

['6', '3', '64%', '67% (38/57)', '44% (14/32)', '43% (3/7)', '38% (21/56)', '74% (17/23)', '63% (5/8)', '27', '31', '61% (25/41)', '9', '58% (52/89)', '48% (38/79)', '54% (90/168)', '3', '67% (8/12)', '42% (5/12)', '54% (13/24)', '0', '1', '71%', '63% (35/56)', '26% (6/23)', '38% (3/8)', '33% (19/57)', '56% (18/32)', '57% (4/7)', '18', '27', '50% (7/14)', '5', '52% (41/79)', '42% (37/89)', '46% (78/168)', '4', '58% (7/12)', '33% (4/12)', '46% (11/24)', 'garin-cristian', 'tien-learner', '06.03.2023 18:35', '2\n-\n0\nFINISHED', 'ATP - SINGLES: INDIAN WELLS (USA) - QUALIFICATION, HARD - SEMI-FINALS', '', '']

47

['6', '3', '64%', '67% (38/57)', '44% (14/32)', '63% (5/8)', '48% (38/79)', '54% (90/168)', '54% (13/24)', '0', '1', '71%', '63% (35/56)', '26% (6/23)', '57% (4/7)', '42% (37/89)', '46% (78/168)', '46% (11/24)', 'garin-cristian', 'tien-learner', '06.03.2023 18:35', '2\n-\n0\nFINISHED', 'ATP - SINGLES: INDIAN WELLS (USA) - QUALIFICATION, HARD - SEMI-FINALS', '', '']

['11', '0', '

['6', '4', '60%', '70% (38/54)', '47% (17/36)', '57% (4/7)', '36% (18/50)', '62% (16/26)', '50% (6/12)', '26', '29', '73% (22/30)', '6', '61% (55/90)', '45% (34/76)', '54% (89/166)', '6', '77% (10/13)', '46% (6/13)', '62% (16/26)', '3', '1', '66%', '64% (32/50)', '38% (10/26)', '50% (6/12)', '30% (16/54)', '53% (19/36)', '43% (3/7)', '23', '40', '54% (19/35)', '7', '55% (42/76)', '39% (35/90)', '46% (77/166)', '2', '54% (7/13)', '23% (3/13)', '38% (10/26)', 'passaro-francesco', 'krueger-mitchell', '06.03.2023 14:10', '2\n-\n1\nFINISHED', 'ATP - SINGLES: INDIAN WELLS (USA) - QUALIFICATION, HARD - SEMI-FINALS', '', '']

47

['6', '4', '60%', '70% (38/54)', '47% (17/36)', '50% (6/12)', '45% (34/76)', '54% (89/166)', '62% (16/26)', '3', '1', '66%', '64% (32/50)', '38% (10/26)', '43% (3/7)', '39% (35/90)', '46% (77/166)', '38% (10/26)', 'passaro-francesco', 'krueger-mitchell', '06.03.2023 14:10', '2\n-\n1\nFINISHED', 'ATP - SINGLES: INDIAN WELLS (USA) - QUALIFICATION, HARD - SEMI-FINALS', '